In [1]:
import os, sys
sys.path.insert(0, '..')

In [2]:
import gensim
import gc
from os.path import join as j
import json
from tqdm import tqdm, trange
from models import glove, custom_trained_model, word2vec
from utils.dataset import PandasDataset
from datasets.nyt import Nyt
from utils.weat import WEAT
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns, numpy as np, pandas as pd, random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')


gc.enable()

In [3]:
def get_bar_plot(y, pred):
    u = np.unique(y)
    classes = len(u)
    match = [0] * classes * 2
    for idx, true in enumerate(y):
        if true == pred[idx]:
            match[true] += 1
        else:
            match[true + classes] += 1
    return pd.DataFrame({'x': list(u) * 2, 'y': match, 'hue': [True] * classes + [False] * classes})

In [4]:
def plot_(x, y, colors, z=None, title="year", three=False, scale=True):
    # https://stackoverflow.com/a/60621783
    sns.set_style("whitegrid", {'axes.grid' : False})
    fig = plt.figure(figsize=(6,6))
    if three:
        ax = Axes3D(fig)
        g = ax.scatter(x, y, z, c=colors, marker='o', depthshade=False, cmap='Paired')
        if scale:
            ax.set_zlim(-1, 1)
            ax.set_xlim(-1, 1)
            ax.set_ylim(-1, 1)
        ax.set_zlabel('Z Label')
        ax.set_xlabel('X Label')
        ax.set_ylabel('Y Label')
        ax.set_title(title)
        legend = ax.legend(*g.legend_elements(), loc="lower center", borderaxespad=-10, ncol=4)
        ax.add_artist(legend)
    else:
        plt.scatter(x, y, c=colors)
        if scale:
            plt.xlim(-2, 2)
            plt.ylim(-2, 2)
        plt.title(title)
    plt.show()

In [5]:
def plot_model(model, model_title, man_words, woman_words, occupations=[]):
    
    vecs = model.transform(man_words + woman_words + occupations)
    colors = ['blue'] * len(man_words) + ['pink'] * len(woman_words) + ['green'] * len(occupations)
    
    # plot 3D graph using PCA
    three = PCA(random_state=0).fit_transform(vecs)[:, :3]
    plot_(three[:, 0], three[:, 1], z=three[:, 2], title=model_title + "_3D_PCA", three=True, scale=False, colors=colors)
    
    # plot 2 D plot using PCA
    two = PCA(random_state=0).fit_transform(vecs)[:, :2]
    plot_(two[:, 0], two[:, 1], title=model_title + "_2D_PCA", three=False, scale=False, colors=colors)
    
    
    # plot LDA
    y = np.concatenate([np.zeros(shape=len(man_words), dtype=int), np.ones(dtype=int, shape=len(woman_words)), 
                    np.full(dtype=int, shape=len(occupations), fill_value=2)]) 
    two = LinearDiscriminantAnalysis().fit_transform(vecs, y)
    
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(vecs, y)
    pred = knn.predict(vecs)
    df = get_bar_plot(y=y, pred=pred)
    if len(occupations):
        plot_(two[:, 0], two[:, 1], title=model_title + "_2D_LDA", three=False, scale=False, colors=colors)
    else:
        plot_(two[:, 0], y, title=model_title + "_1D_LDA", three=False, scale=False, colors=colors)
    sns.barplot(data=df, x='x', y='y', hue='hue', ).set_title(model_title + "_KNN classification")
    plt.show()
    return WEAT(model, words_json='../weat/weat.json').get_scores()

In [6]:
DIR = '/tmp/temp/'
YEARS = range(1921, 2021, 10)
EMBEDDINGS_DIR = '../trained_models/glove/embeddings_{}/'
MAN, WOMAN, OCCUPATIONS = 'Man words', 'Woman words', 'Occupations with Human Stereotype Scores'
words = json.load(open('../weat/GargWordList.json'))
words[WOMAN].remove('femen') # not sure what femen is !! 
CSV = "df_{st}_to_{end}.csv"
SCORES = np.zeros(shape=(len(YEARS), 7))
CUSTOM_MODEL_PATH = "../trained_models/GoogleNews-vectors-negative300.bin"

# GLOVE MODELS

In [7]:
for idx, y in tqdm(enumerate(YEARS)):
    dataset = DIR + CSV.format(st=y, end=y+9)
    saved_model_path = EMBEDDINGS_DIR.format(y)
    lines = Nyt(dataset).lines
    m = glove.Glove(load=False).fit(lines, glove_path='../scripts/').save(saved_model_path)

0it [00:00, ?it/s]

make: Entering directory '/home/ashutosh/study/bias_manifold/scripts'
rm -rf glove shuffle cooccur vocab_count build
make: Leaving directory '/home/ashutosh/study/bias_manifold/scripts'
make: Entering directory '/home/ashutosh/study/bias_manifold/scripts'
mkdir -p build
gcc ./glove.c -o build/glove -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic


./glove.c: In function ‘save_params’:
./glove.c:249:50: warning: format ‘%ld’ expects argument of type ‘long int’, but argument 3 has type ‘long long int’ [-Wformat=]
  249 |               if (write_header) fprintf(fout, "%ld %d\n", vocab_size, vector_size);
      |                                                ~~^        ~~~~~~~~~~
      |                                                  |        |
      |                                                  long int long long int
      |                                                %lld
./glove.c: In function ‘initialize_parameters’:
./glove.c:98:13: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
   98 |             fread(&W[a], sizeof(real), 1, fin);
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./glove.c: In function ‘glove_thread’:
./glove.c:133:9: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  133 |       

gcc ./shuffle.c -o build/shuffle -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
gcc ./cooccur.c -o build/cooccur -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic


./shuffle.c: In function ‘shuffle_merge’:
./shuffle.c:107:17: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  107 |                 fread(&array[i], sizeof(CREC), 1, fid[j]);
      |                 ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./shuffle.c: In function ‘shuffle_by_chunks’:
./shuffle.c:165:9: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  165 |         fread(&array[i], sizeof(CREC), 1, fin);
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./cooccur.c: In function ‘merge_files’:
./cooccur.c:267:9: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  267 |         fread(&new, sizeof(CREC), 1, fid[i]);
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./cooccur.c:277:5: warning: ignoring return value of ‘fread’ declared with attribute ‘warn_unused_result’ [-Wunused-result]
  277 |     fread(&new, 

...

gcc ./vocab_count.c -o build/vocab_count -lm -pthread -Ofast -march=native -funroll-loops -Wall -Wextra -Wpedantic
make: Leaving directory '/home/ashutosh/study/bias_manifold/scripts'


BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

Processing token: 0100000200000300000400000500000600000700000800000900000100000011000001200000130000014000001500000160000017000001800000190000020000002100000220000023000002400000250000026000002700000280000029000003000000310000032000003300000340000035000003600000370000038000003900000400000041000004200000430000044000004500000460000047000004800000490000050000005100000520000053000005400000550000056000005700000580000059000006000000610000062000006300000640000065000006600000670000068000006900000700000071000007200000730000074000007500000760000077000007800000790000080000008100000820000083000008400000850000086000008700000880000089000009000000910000092000009300000940000095000009600000970000098000009900000100000001010000010200000103000001040000010500000106000001070000010800000109000001100000011100000112000001130000011400000115000001160000011700000118000001190000012000000121000001220000012300000124000001250000012600000127000001280000012900000130000001310000013200000133000001340000013500000136000001

Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.1800000 lines.1900000 lines.2000000 lines.2100000 lines.2200000 lines.2300000 lines.2400000 lines.2500000 lines.2600000 lines.2700000 lines.2800000 lines.2900000 lines.3000000 lines.3100000 lines.3200000 lines.3300000 lines.3400000 lines.3500000 lines.3600000 lines.3700000 lines.3800000 lines.3900000 lines.4000000 lines.4100000 lines.4200000 lines.4300000 lines.4400000 lines.4500000 lines.4600000 lines.4700000 lines.4800000 lines.4900000 lines.5000000 lines.5100000 lines.5200000 lines.5300000 lines.5400000 lines.5500000 lines.5600000 lines.5700000 lines.5800000 lines.5900000 lines.6000000 lines.6100000 lines.6200000 lines.6300000 lines.6400000 lines.6500000 lines.6600000 lines.6700000 lines.6800000 lines.6900000 lin

41800000 lines.41900000 lines.42000000 lines.42100000 lines.42200000 lines.42300000 lines.42400000 lines.42500000 lines.42600000 lines.42700000 lines.42800000 lines.42900000 lines.43000000 lines.43100000 lines.43200000 lines.43300000 lines.43400000 lines.43500000 lines.43600000 lines.43700000 lines.43800000 lines.43900000 lines.44000000 lines.44100000 lines.44200000 lines.44300000 lines.44400000 lines.44500000 lines.44600000 lines.44700000 lines.44800000 lines.44900000 lines.45000000 lines.45100000 lines.45200000 lines.45300000 lines.45400000 lines.45500000 lines.45600000 lines.45700000 lines.45800000 lines.45900000 lines.46000000 lines.46100000 lines.46200000 lines.46300000 lines.46400000 lines.46500000 lines.46600000 lines.46700000 lines.46800000 lines.46900000 lines.47000000 lines.47100000 lines.47200000 lines.47300000 lines.47400000 lines.47500000 lines.47600000 lines.47700000 lines.47800000 lines.47900000 lines.48000000 lines.48100000 lines.48200000 lines.48300000 lines.Merging co